In [14]:
import pandas as pd
from datetime import datetime, timedelta
import boto3
import os
import io

In [7]:
service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
access_key = ' '
secret_key = ' '
bucket_name = 'bucket1chaegpt'

### 파일 가져오기

In [2]:
kr_list = pd.read_csv('국내_산업분류(최종).csv', encoding = 'cp949')['기업'].unique()
kr_list

array(['탑머티리얼', '삼성전자', 'SK하이닉스', ..., '테고사이언스', 'HLB생명과학', '바이오솔루션'],
      dtype=object)

In [3]:
#주가 데이터 클라우드에서 불러오기
def download_stock_data(bucket_name, company_name, local_file_path):
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)
    
    file_name = f'{company}_주가데이터.xlsx'
    object_name = f'kr_stock_crawling/{file_name}'

    s3.download_file(bucket_name, object_name, local_file_path)

In [4]:
#평균 등략률 계산하기 (3개월)
def average_change(data):
    today = datetime.today()
    
    one_month = data[(today - data['Date']).dt.days <= 30]['Change'].mean()
    two_month = data[(today - data['Date']).dt.days <= 60]['Change'].mean()
    three_month = data[(today - data['Date']).dt.days <= 90]['Change'].mean()

    result_df = pd.DataFrame({'기간' : ['1개월', '2개월', '3개월'],
                              '평균 등락률' : [one_month, two_month, three_month]})
    
    return result_df

In [5]:
#클라우드에 업로드
def upload(company_name, data, s3_client, bucket_name, endpoint_url, access_key, secret_key):

    # 데이터프레임을 파일로 저장
    xlsx_filename = f'{company_name}_평균등락률.xlsx'
    xlsx_buffer = io.BytesIO()
    data.to_excel(xlsx_buffer, index = False)
    xlsx_buffer.seek(0)

    # xlsx 파일 클라우드 업로드
    object_name = f'change_average_kr/{xlsx_filename}'
    s3_client.upload_fileobj(xlsx_buffer, bucket_name, object_name)
    print(f"Uploaded {xlsx_filename} to S3")

In [15]:
for company in kr_list:
    file_path = f'C:/Users/pc/미래에셋/국내_주가데이터/{company}_주가데이터.xlsx'
    
    # 데이터 가져오기
    download_stock_data(bucket_name, company, file_path)
    
    # 데이터프레임으로 읽어오기
    st_data = pd.read_excel(file_path)
    
    result_df = average_change(st_data)
    result_df['Company'] = company
    
    # 파일 삭제
    if os.path.exists(file_path):
        os.remove(file_path)
    else :
        print(f"파일이 존재하지 않습니다: {file_path}")
        
    # AWS S3 클라이언트 생성
    s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                      aws_secret_access_key=secret_key)

    # 클라우드 폴더 생성
    object_name = 'change_average_kr/'
    s3.put_object(Bucket=bucket_name, Key=object_name)

    # 주가 데이터 수집 및 파일로 저장
    upload(company, result_df, s3, bucket_name, endpoint_url, access_key, secret_key)

Uploaded 탑머티리얼_차트분석.xlsx to S3
